In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set some display options for better viewing
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)


In [6]:
customers = pd.read_csv('/Users/shivamkumar/laptop/H2h/data/customers.csv')
transactions = pd.read_csv('/Users/shivamkumar/laptop/H2h/data/transactions.csv')
support_tickets = pd.read_csv('/Users/shivamkumar/laptop/H2h/data/support_tickets.csv')

In [7]:
# --- Step 1: Handle Missing Values in customers DataFrame ---

# Calculate the medians
median_minutes = customers['monthly_minutes'].median()
median_data_usage = customers['data_usage_gb'].median()

print(f"Median for 'monthly_minutes': {median_minutes}")
print(f"Median for 'data_usage_gb': {median_data_usage}\n")

# Fill the missing values using the calculated medians
customers['monthly_minutes'].fillna(median_minutes, inplace=True)
customers['data_usage_gb'].fillna(median_data_usage, inplace=True)

# Verify that the missing values have been filled
print("--- Missing values in 'customers' after filling ---")
print(customers.isnull().sum())

Median for 'monthly_minutes': 242.0
Median for 'data_usage_gb': 12.33

--- Missing values in 'customers' after filling ---
customer_id                   0
age                           0
gender                        0
city                          0
state                         0
tenure_months                 0
contract_type                 0
payment_method                0
monthly_charges               0
total_charges                 0
monthly_minutes               0
data_usage_gb                 0
num_products                  0
feature_adoption_score        0
last_login_days_ago           0
logins_last_month             0
customer_satisfaction         0
account_status                0
churned                       0
churn_date                42500
dtype: int64


/var/folders/pv/8kgk_c4n7r1g_vxl8bhc3qkr0000gn/T/ipykernel_44927/3901054759.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  customers['monthly_minutes'].fillna(median_minutes, inplace=True)
/var/folders/pv/8kgk_c4n7r1g_vxl8bhc3qkr0000gn/T/ipykernel_44927/3901054759.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are 

In [8]:
# --- Step 2: Convert 'transaction_date' to a datetime object ---

print("--- Data type of 'transaction_date' BEFORE conversion ---")
print(transactions['transaction_date'].dtype)

# Convert the column to datetime
transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'])

print("\n--- Data type of 'transaction_date' AFTER conversion ---")
print(transactions['transaction_date'].dtype)

# Let's also check the first 5 rows to see the change
print("\n--- First 5 rows of transactions DataFrame after conversion ---")
print(transactions.head())

--- Data type of 'transaction_date' BEFORE conversion ---
object

--- Data type of 'transaction_date' AFTER conversion ---
datetime64[ns]

--- First 5 rows of transactions DataFrame after conversion ---
  transaction_id customer_id transaction_date  amount payment_method transaction_type   status
0    TXN00000001  CUST000001       2022-01-18   71.15   Mailed Check  Initial Payment  Success
1    TXN00000002  CUST000001       2022-02-18   72.62   Mailed Check     Subscription  Success
2    TXN00000003  CUST000001       2022-03-22   71.91   Mailed Check     Subscription  Success
3    TXN00000004  CUST000001       2022-04-17   73.44   Mailed Check     Subscription  Success
4    TXN00000005  CUST000001       2022-05-17   72.81   Mailed Check     Subscription  Success


In [9]:
# --- Step 3: Create Transaction-Based Features ---

# Calculate a recent date to determine 'days_since_last_transaction'
recent_date = transactions['transaction_date'].max()
print(f"The most recent transaction date is: {recent_date.date()}\n")

# Group by customer_id and aggregate to create new features
transaction_features = transactions.groupby('customer_id').agg(
    total_transactions=('transaction_id', 'count'),
    total_failed_transactions=('status', lambda x: (x == 'Failed').sum()),
    avg_transaction_amount=('amount', 'mean'),
    days_since_last_transaction=('transaction_date', lambda x: (recent_date - x.max()).days)
).reset_index()

# Calculate the failed transaction rate
transaction_features['failed_transaction_rate'] = (
    transaction_features['total_failed_transactions'] / transaction_features['total_transactions']
)

# Display the first 5 rows of our new features DataFrame
print("--- Newly created transaction features (first 5 rows) ---")
print(transaction_features.head())

The most recent transaction date is: 2024-12-06

--- Newly created transaction features (first 5 rows) ---
  customer_id  total_transactions  total_failed_transactions  avg_transaction_amount  days_since_last_transaction  failed_transaction_rate
0  CUST000001                  36                          1               72.334722                            5                 0.027778
1  CUST000002                  13                          1               89.181538                           64                 0.076923
2  CUST000003                  18                          1               66.158889                            2                 0.055556
3  CUST000004                   6                          0               63.996667                            2                 0.000000
4  CUST000005                  34                          0               93.252059                          600                 0.000000


In [10]:
# --- Step 4: Create Support Ticket-Based Features ---

# Fill missing resolution times with a high number to signify "still open"
support_tickets['resolution_time_hours'].fillna(999, inplace=True)

# Group by customer_id and aggregate
support_features = support_tickets.groupby('customer_id').agg(
    total_tickets=('ticket_id', 'count'),
    open_tickets=('status', lambda x: (x == 'Open').sum()),
    high_priority_tickets=('priority', lambda x: x.isin(['High', 'Critical']).sum()),
    avg_resolution_time=('resolution_time_hours', 'mean')
).reset_index()

# Display the first 5 rows of our new support features DataFrame
print("--- Newly created support ticket features (first 5 rows) ---")
print(support_features.head())

/var/folders/pv/8kgk_c4n7r1g_vxl8bhc3qkr0000gn/T/ipykernel_44927/1708167249.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  support_tickets['resolution_time_hours'].fillna(999, inplace=True)


--- Newly created support ticket features (first 5 rows) ---
  customer_id  total_tickets  open_tickets  high_priority_tickets  avg_resolution_time
0  CUST000001             17             2                      6           145.882353
1  CUST000002              4             0                      1            21.250000
2  CUST000003             18             6                      3           348.388889
3  CUST000004              3             1                      2           350.000000
4  CUST000005             14             2                      6           164.500000


In [11]:
print("----------- Customers DataFrame Info (after cleaning) -----------")
customers.info()
print("\n----------- Transactions DataFrame Info (after cleaning) -----------")
transactions.info()
print("\n----------- Support Tickets DataFrame Info (after feature engineering prep) -----------")
# We already filled the missing values in the previous step, let's just confirm
support_tickets.info()

print("\n----------- Our NEW transaction_features DataFrame -----------")
transaction_features.info()

print("\n----------- Our NEW support_features DataFrame -----------")
support_features.info()

----------- Customers DataFrame Info (after cleaning) -----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customer_id             50000 non-null  object 
 1   age                     50000 non-null  int64  
 2   gender                  50000 non-null  object 
 3   city                    50000 non-null  object 
 4   state                   50000 non-null  object 
 5   tenure_months           50000 non-null  int64  
 6   contract_type           50000 non-null  object 
 7   payment_method          50000 non-null  object 
 8   monthly_charges         50000 non-null  float64
 9   total_charges           50000 non-null  float64
 10  monthly_minutes         50000 non-null  float64
 11  data_usage_gb           50000 non-null  float64
 12  num_products            50000 non-null  int64  
 13  feature_adoption_score  5

In [12]:
# --- Step 5: Merge, Clean, and Encode for the Final Dataset ---

# Start with the main customers DataFrame
df_model = customers.copy()

# 1. MERGE DATAFRAMES
print("--- Merging transaction features...")
df_model = pd.merge(df_model, transaction_features, on='customer_id', how='left')

print("--- Merging support ticket features...")
df_model = pd.merge(df_model, support_features, on='customer_id', how='left')


# 2. FILL NaNs CREATED BY MERGES
# These NaNs mean the customer had 0 transactions or 0 tickets, so we fill with 0.
df_model.fillna(0, inplace=True)


# 3. ENCODE CATEGORICAL FEATURES & DROP UNNECESSARY COLUMNS

# One-Hot Encode nominal features (no natural order)
df_model = pd.get_dummies(df_model, columns=['contract_type', 'payment_method'], prefix=['contract', 'payment'])

# Label Encode binary features like gender
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_model['gender_encoded'] = le.fit_transform(df_model['gender'])


# Finally, drop columns that are identifiers, high-cardinality text, or would leak the answer
columns_to_drop = [
    'customer_id',
    'gender',          # We have gender_encoded now
    'city',            # Too many unique values
    'state',           # Too many unique values
    'account_status',  # This would leak the churn information
    'churn_date'       # This is the answer, not a feature
]
df_model.drop(columns=columns_to_drop, inplace=True, errors='ignore')


# --- Final Verification ---
print("\n\n--- Final, Model-Ready DataFrame (first 5 rows) ---")
print(df_model.head())
print("\n--- Info of Model-Ready DataFrame ---")
df_model.info()

--- Merging transaction features...
--- Merging support ticket features...


--- Final, Model-Ready DataFrame (first 5 rows) ---
   age  tenure_months  monthly_charges  total_charges  monthly_minutes  data_usage_gb  num_products  feature_adoption_score  last_login_days_ago  logins_last_month  customer_satisfaction  churned  total_transactions  total_failed_transactions  avg_transaction_amount  days_since_last_transaction  failed_transaction_rate  total_tickets  open_tickets  high_priority_tickets  avg_resolution_time  contract_Month-to-Month  contract_One Year  contract_Two Year  payment_Bank Transfer  payment_Credit Card  payment_Electronic Check  payment_Mailed Check  gender_encoded
0   49             36            72.11        2658.82             73.0          63.75             1                    68.6                    3                 13                    3.0        0                  36                          1               72.334722                            5           

In [13]:
# --- Phase 3, Step 1: Save the Cleaned DataFrame to a CSV file ---

# Create a new directory named 'processed_data' if it doesn't exist
import os
os.makedirs('processed_data', exist_ok=True)

# Save the DataFrame
df_model.to_csv('processed_data/cleaned_customer_churn_data.csv', index=False)

print("Cleaned data has been saved to 'processed_data/cleaned_customer_churn_data.csv'")

Cleaned data has been saved to 'processed_data/cleaned_customer_churn_data.csv'
